In [1]:
using JuMP;
using Clp;
using Distributions;

# srand(123456) # Setting the seed

# parameters
N_seq = [100 1000 10000];

for i=1:length(N_seq)
    N = N_seq[i]
    println("========================================================================");
    distrb1 = Uniform(-0.8, 0.8);
    distrb2 = Normal(0, 12);
    distrb3 = Exponential(2.5);
    distrb4 = Normal(0, 9);

    xi1 = rand(distrb1, N);
    xi2 = rand(distrb2, N);
    xi3 = rand(distrb3, N);
    xi4 = rand(distrb4, N);

    Master=Model(solver=ClpSolver());

    @variable(Master, x[1:2]>=0);
    @variable(Master, theta>=0);

    @constraint(Master, x[1]+x[2] <= 100);
    @objective(Master, Min, 2*x[1]+3*x[2]+theta);

    # check if there need feasible cut
    status = solve(Master);
    println(status);
    xval = getvalue(x);

    ## Construct 2nd stage problem
    Subprob = Model(solver=ClpSolver());
    @variable(Subprob, y[1:2]>=0);
    @objective(Subprob, Min, 7*y[1]+12*y[2]);
    # change the RHS later
    @constraint(Subprob, constr1, y[1]>=0);
    @constraint(Subprob, constr2, y[2]>=0);

    LB=0;
    UB=1e10;
    iter=0;

    while (UB-LB)*1.0/UB > 1e-5
    # while iter <= 3
        iter = iter + 1;  
        # First solve the master problem, and get x value
        status = solve(Master);
        xval = getvalue(x);
        
        thetaval = getvalue(theta);
        LB = getobjectivevalue(Master);
        
        duals = zeros(2);
        tempUB = 0;
        tempUB = tempUB + 2*xval[1] + 3*xval[2];
        
        constant = 0;
        x1coef = 0;
        x2coef = 0;
        
        for k=1:N
            JuMP.setRHS(constr1, 180+xi2[k]-((2+xi1[k])*xval[1]+6*xval[2]));
            JuMP.setRHS(constr2, 162+xi4[k]-(3*xval[1]+(3.4-xi3[k])*xval[2]));

            status = solve(Subprob);
            tempUB = tempUB + getobjectivevalue(Subprob)*1.0/N;
            duals[1] = getdual(constr1);
            duals[2] = getdual(constr2);
            
            x1coef = x1coef + (2+xi1[k])*duals[1]+3*duals[2];
            x2coef = x2coef + 6*duals[1]+(3.4-xi3[k])*duals[2];
            constant = constant + (180+xi2[k])*duals[1] + (162+xi4[k])*duals[2];
        end
        
        if tempUB < UB
            UB = tempUB;
        end
        
        if (UB-LB)*1.0/UB < 1e-5
            break;
        else
            # If not converge, MUST be able to generate a cut
            # Now construct cut:
            # Cut should look like: theta >= sum_k p_k(xi[k]-x)*dual[k]
            
            # comment out to see the added constraints
            @constraint(Master,theta >= constant*1.0/N - (x1coef*1.0/N)*x[1] - (x2coef*1.0/N)*x[2]);
            @printf "OPT: x1coef = %f, x2coef = %f, cutrhs = %f\n" (x1coef*1.0/N) (x2coef*1.0/N) (constant*1.0/N);
        end
    end

    println("=================== FINAL RESULTS on ", N_seq[i]," =================\n");
    println("LB = ", LB);
    println("UB = ", UB);
    println("xval1 = ", xval[1]);
    println("xval2 = ", xval[2]);
end

Optimal


OPT: x1coef = 49.556501, x2coef = 52.645276, cutrhs = 3220.890819
OPT: x1coef = 12.980970, x2coef = 40.740000, cutrhs = 1232.932847
OPT: x1coef = 18.381190, x2coef = 11.062262, cutrhs = 1303.612398
OPT: x1coef = 7.941085, x2coef = 13.262741, cutrhs = 748.079006
OPT: x1coef = 7.200000, x2coef = -8.463602, cutrhs = 383.110528
OPT: x1coef = 4.240696, x2coef = -1.886270, cutrhs = 318.682003
OPT: x1coef = 2.956909, x2coef = 9.761003, cutrhs = 358.385191
OPT: x1coef = 2.440696, x2coef = 0.707607, cutrhs = 220.571404
OPT: x1coef = 1.098116, x2coef = 5.040000, cutrhs = 158.099649
OPT: x1coef = 2.457186, x2coef = 5.384227, cutrhs = 274.958937
OPT: x1coef = 1.360696, x2coef = 3.461003, cutrhs = 164.400564
OPT: x1coef = 2.290852, x2coef = 2.430665, cutrhs = 228.978646
OPT: x1coef = 1.930852, x2coef = 3.284227, cutrhs = 209.488113
OPT: x1coef = 1.720696, x2coef = 2.444227, cutrhs = 182.716962
OPT: x1coef = 1.818116, x2coef = 2.864227, cutrhs = 195.546344
=================== FINAL RESULTS on 100 ==